In [2]:
import os
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD

In [4]:
# Directories for dataset
train_dir = 'cropped_imgs/train'
test_dir = 'cropped_imgs/test'
eval_dir = 'cropped_imgs/val'

In [6]:
# Image size for resizing
img_size = (224, 224)

In [8]:
# Step 1: Data Augmentation
datagen_train = ImageDataGenerator(
    rescale=1./255,  # Normalize the images
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [10]:
datagen_test = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data

train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',  # Use 'binary' if it's a binary classification task
    shuffle=True
)


Found 2514 images belonging to 18 classes.


In [12]:
test_generator = datagen_test.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',  # Use 'binary' if it's a binary classification task
    shuffle=False
)

Found 719 images belonging to 18 classes.


In [14]:
# Step 2: Choose a pre-trained model (EfficientNetB0 for better performance)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers for initial training
base_model.trainable = False


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [16]:
# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),  # Regularization to avoid overfitting
    layers.Dense(train_generator.num_classes, activation='softmax')  # Adjust for number of classes
])

In [18]:
# Compile the model using a powerful optimizer (AdamW)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
# Step 3: Callbacks for Early Stopping and Learning Rate Scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [22]:
# Step 4: Training the Model
history = model.fit(
    train_generator,
    epochs=50,  # Keep epochs high for full training
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

E:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 63s 661ms/step - accuracy: 0.1019 - loss: 2.8475 - val_accuracy: 0.1099 - val_loss: 2.8067 - learning_rate: 1.0000e-04
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 611ms/step - accuracy: 0.1126 - loss: 2.8205 - val_accuracy: 0.1099 - val_loss: 2.8040 - learning_rate: 1.0000e-04
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 603ms/step - accuracy: 0.1255 - loss: 2.8050 - val_accuracy: 0.1099 - val_loss: 2.8008 - learning_rate: 1.0000e-04
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 606ms/step - accuracy: 0.1178 - loss: 2.8116 - val_accuracy: 0.1099 - val_loss: 2.7997 - learning_rate: 1.0000e-04
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 606ms/step - accuracy: 0.1106 - loss: 2.8147 - val_accuracy: 0.1099 - val_loss: 2.8025 - learning_rate: 1.0000e-04
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 604ms/step - accuracy: 0.1292 - loss: 2.7995 - val_accuracy: 0.1099 - val_loss: 2.8001 - learning_rate: 1.0000e-04
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 609ms/step - acc

In [24]:
# Step 5: Fine-tune the base model
base_model.trainable = True  # Unfreeze base model layers
for layer in base_model.layers[:100]:  # Fine-tune top 100 layers
    layer.trainable = False  # Freeze the first few layers

In [26]:
# Re-compile the model to apply changes
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
# Train again to fine-tune the model
history_finetune = model.fit(
    train_generator,
    epochs=50,  # Continue training for additional epochs
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 91s 927ms/step - accuracy: 0.0496 - loss: 2.9733 - val_accuracy: 0.1057 - val_loss: 2.8160 - learning_rate: 1.0000e-05
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 931ms/step - accuracy: 0.1035 - loss: 2.8704 - val_accuracy: 0.0987 - val_loss: 2.8549 - learning_rate: 1.0000e-05
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 940ms/step - accuracy: 0.1047 - loss: 2.8227 - val_accuracy: 0.0334 - val_loss: 2.8730 - learning_rate: 1.0000e-05
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 940ms/step - accuracy: 0.1227 - loss: 2.8160 - val_accuracy: 0.1168 - val_loss: 2.8620 - learning_rate: 1.0000e-05
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 931ms/step - accuracy: 0.1291 - loss: 2.7611 - val_accuracy: 0.1224 - val_loss: 2.8502 - learning_rate: 1.0000e-05
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 940ms/step - accuracy: 0.1223 - loss: 2.7731 - val_accuracy: 0.1433 - val_loss: 2.8238 - learning_rate: 1.0000e-05
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 932ms/step - acc

In [30]:
# Step 6: Evaluate the model
eval_generator = datagen_test.flow_from_directory(
    eval_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',  # Use 'binary' if it's a binary classification task
    shuffle=False
)


Found 346 images belonging to 18 classes.


In [32]:
eval_loss, eval_acc = model.evaluate(eval_generator)
print(f"Final Evaluation Loss: {eval_loss}, Final Evaluation Accuracy: {eval_acc}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 356ms/step - accuracy: 0.2201 - loss: 2.6270
Final Evaluation Loss: 2.690399646759033, Final Evaluation Accuracy: 0.17052023112773895
